**Домашняя работа №1 - Введение в Feature Engineering**

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Придумаем искусственный датасет про параметры тела человека.
Возьмем такие признаки: возраст (age) от 18 лет, вес (weight), рост (height) и будем предсказывать условное количество калорий в день, которые потребляет данный человек (kkal) (для данного датасета рассчитаем исходя из роста и веса)

In [16]:
# Создаём сэмпл
n_samples = 600

age = np.random.choice(80, n_samples) + 18
weight = np.random.choice(200, n_samples) + 50
height = np.random.choice(70, n_samples) + 150

kkal = 80 * weight / (height / 100) ** 2

data = pd.DataFrame({'age': age, 'weight': weight, 'height': height, 'kkal': kkal})
data.head(5)

,age,weight,height,kkal
0,84,204,155,6792.924037
1,21,61,157,1979.796341
2,45,122,187,2791.043496
3,18,137,213,2415.746435
4,29,109,180,2691.358025


Попробуем предсказать "в лоб". Возьмем все данные как есть:

In [17]:
from sklearn.metrics import mean_absolute_error

X = data[['age', 'weight', 'height']]
y = data['kkal']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['age', 'weight', 'height']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  0.8453671   24.64296397 -39.9083044 ]
Bias: 7259.403864072254
Error: 261.1928698874553


Ошибка получилась очень большая. Возраст здесь явно лишний - попробуем его убрать (хотя в жизни количество потребляемой еды нередко зависит от возраста, но мы этим пренебрежем, у нас ведь учебный датасет:))

In [18]:
X = data[['weight', 'height']]
y = data['kkal']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['weight', 'height']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 24.6580411  -39.94476941]
Bias: 7312.99190021363
Error: 261.0513891047995


Ошибка изменилась совсем немного.
Попробуем ввести параметр индекс массы тела ( вес / квадрат роста в метрах) - index_m

In [19]:
# Создаем новый признак
data['index_m'] = data['weight'] / (data['height'] / 100) ** 2
data.head(5)

,age,weight,height,kkal,index_m
0,84,204,155,6792.924037,84.911550
1,21,61,157,1979.796341,24.747454
2,45,122,187,2791.043496,34.888044
3,18,137,213,2415.746435,30.196830
4,29,109,180,2691.358025,33.641975


Снова рассчитаем нашу модель, теперь уже используя только новый признак:

In [20]:
X = data[['index_m']]
y = data['kkal']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['index_m']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [80.]
Bias: -4.547473508864641e-13
Error: 4.964325247177233e-13


Итак, ошибка значительно сократилась. Теперь мы убедились в том, что в некоторых случаях если использовать признак, созданный из перемножения других признаков, то качество предсказания линейной модели значительно повышается.